In [1]:
# default_exp models.resnet2

In [2]:
# export
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score

import os
import argparse
import numpy as np

In [3]:
# export
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class _ResNet(nn.Module):
    def __init__(self, block, num_blocks, input_size=64, num_classes=10):
        super(_ResNet, self).__init__()
        self.in_planes = input_size

        self.conv1 = nn.Conv2d(3, input_size, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(input_size)
        self.layer1 = self._make_layer(block, input_size, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        #self.linear = nn.Linear(512*block.expansion, num_classes)
        self.linear = nn.Linear(512*4, num_classes)
        
        print("512*block.expansion", 512*block.expansion)
        print("num_classes", num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        embeds = F.avg_pool2d(out, 4)
        embeds = embeds.view(embeds.size(0), -1)
        last = self.linear(embeds)
        return last, embeds


In [4]:
# export
class _CenterLoss(nn.Module):
    """Center loss.
    
    Reference:
    Wen et al. A Discriminative Feature Learning Approach for Deep Face Recognition. ECCV 2016.
    
    Args:
        num_classes (int): number of classes.
        feat_dim (int): feature dimension.
    """
    def __init__(self, num_classes, feat_dim):
        super(_CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))
        self.classes = torch.arange(self.num_classes).long().cuda() # HACK PASADO DE LANZA

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
        batch_size = x.size(0)
        
        distmat = torch.pow(x, 2).sum(dim=1, 
                                      keepdim=True).expand(batch_size, 
                                                           self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, 
                                                 keepdim=True).expand(self.num_classes, 
                                                                      batch_size).t()

        distmat = torch.addmm(input=distmat, 
                              mat1=x, 
                              mat2=self.centers.t(), 
                              beta=1, 
                              alpha=-2)
        
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        
        mask = labels.eq(self.classes.expand(batch_size, self.num_classes))

        dist = distmat * mask.float()
        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size

        return loss

In [5]:
# export
class ResNet(object):
    def __init__(self, params):
        
        super().__init__()
        self.params = params
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Instantiate model
        torch.manual_seed(params['random_seed'])
        np.random.seed(params['random_seed'])
        self.model = _ResNet(block=BasicBlock, 
                             num_blocks=[2, 2, 2, 2],
                             num_classes=params['n_classes'],
                             input_size=params['input_size']).to(self.device)
        
        self.centroids = _CenterLoss(num_classes=params['n_classes'], 
                                     feat_dim=512*4).to(self.device)

    def adjust_lr(self, optimizer, lr_decay):
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * lr_decay

    def save_weights(self, path):
        torch.save(self.model.state_dict(), path)    

    def load_weights(self, path):
        self.model.load_state_dict(torch.load(path,
                                          map_location=torch.device(self.device)))
        self.model.eval()

    def fit(self, train_loader, val_loader, vrf_loader):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.model.parameters(), lr=self.params['initial_lr'],
                              momentum=0.9, weight_decay=self.params['weight_decay'])
        coptimizer = optim.SGD(self.centroids.parameters(), lr=self.params['initial_clr'], momentum=0.9)

        # Initialize counters and trajectories
        step = 0
        epoch = 0
        self.best_acc = -1
        metric_trajectories = {'step':  [],
                               'epoch':  [],
                               'train_loss': [],
                               'train_acc': [],
                               'val_loss': [],
                               'val_acc': []}

        print('\n'+'='*37+' Fitting  ResNet '+'='*37)
        while step <= self.params['iterations']:
            # Train
            epoch += 1
            self.model.train()
            for batch_idx, (inputs, targets) in enumerate(train_loader):
                step+=1
                if step > self.params['iterations']:
                    continue
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                optimizer.zero_grad()
                coptimizer.zero_grad()
                
                outputs, embeds = self.model(inputs)
                loss = criterion(outputs, targets) + \
                        self.params['alpha'] * self.centroids(embeds, targets)
                loss.backward()
                
                
                # weird lr=1 for weird chain rule
                for p in self.centroids.parameters():
                    p.grad.data *= (1./self.params['alpha'])
                
                coptimizer.step()
                optimizer.step()

                # Evaluate metrics
                if (step % self.params['display_step']) == 0:
                    train_loss, train_acc = self.evaluate_performance(loader=train_loader, criterion=criterion)
                    #train_loss, train_acc = 0, 0
                    val_loss, val_acc = self.evaluate_performance(loader=val_loader, criterion=criterion)
                    roc, _ = self.evaluate_roc(loader=vrf_loader)
                    
                    display_str = f'step: {step}\t train_loss: {train_loss:.4f} train_acc {train_acc:.2f}'
                    display_str += f'\t val_loss: {val_loss:.4f} val_acc: {val_acc:.2f} roc {roc:.2f}'
                    print(display_str)
                    
                    if val_acc > self.best_acc:
                        print('Saving..')
                        state = {
                            'net': self.model.state_dict(),
                            'acc': val_acc,
                            'epoch': epoch,
                        }
                        if not os.path.isdir('checkpoint'):
                            os.mkdir('checkpoint')
                        torch.save(state, f"./checkpoint/{self.params['experiment_id']}_ckpt.pth")
                        self.best_acc = val_acc
                
                # Update optimizer learning rate
                if step % self.params['adjust_lr_step'] == 0 and step > 0:
                    self.adjust_lr(optimizer=optimizer, lr_decay=self.params['lr_decay'])

        # Check finish condition
        # TODO: check if epoch is interruptable within
        # if step==self.iterations: break
        print('\n'+'='*35+' Finished Train '+'='*35)
        self.train_loss   = metric_trajectories['train_loss'][-1]
        self.train_acc    = metric_trajectories['train_acc'][-1]
        self.val_loss     = metric_trajectories['val_loss'][-1]
        self.val_acc      = metric_trajectories['val_acc'][-1]
        self.trajectories = metric_trajectories
        
    def evaluate_performance(self, loader, criterion):
        self.model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(loader):
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs, _ = self.model(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

        # Save checkpointorch.
        acc = 100.*correct/total
        test_loss = test_loss/total
            
        self.model.train()
        return test_loss, acc
    
    def evaluate_roc(self, loader):
        self.model.eval()
                
        embeds0 = []
        embeds1 = []
        targets = []

        with torch.no_grad():
            for batch_idx, (img0, img1, target) in enumerate(loader):
                img0 = img0.to(self.device)
                img1 = img1.to(self.device)
                
                embeds0 += [self.model(img0)[1].cpu().numpy()]
                embeds1 += [self.model(img1)[1].cpu().numpy()]
                targets += [np.expand_dims(target.cpu().numpy(), 1)]
                
        embeds0  = np.vstack(embeds0)
        embeds1  = np.vstack(embeds1)
        targets  = np.vstack(targets)

        sim_score = np.sum((embeds0 - embeds1) ** 2, axis=1, keepdims=True)        
        roc = roc_auc_score(y_true=targets, y_score=sim_score)
        
        self.model.train()
        return roc, sim_score

SyntaxError: invalid syntax (<ipython-input-5-ec95626c816a>, line 95)

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

params = {'n_classes': 10,
          'iterations': 50_000, #(n_samples/batch_size) * epochs
          'display_step': 2_000,
          'initial_lr': 0.1,
          'lr_decay': 0.5,
          'adjust_lr_step': 50_000//2,
          'weight_decay': 5e-4,
          'random_seed': 1}

model = ResNet(params)
model.fit(train_loader=trainloader, val_loader=testloader)

Files already downloaded and verified
Files already downloaded and verified
512*block.expansion 512
num_classes 10

===================================== Fitting  ResNet =====================================
step: 2000	 train_loss: 0.0064 train_acc 71.30	 val_loss: 0.0087 val_acc: 71.10
step: 4000	 train_loss: 0.0078 train_acc 68.93	 val_loss: 0.0100 val_acc: 69.41


KeyboardInterrupt: 